In [57]:
# import data 
import pandas as pd
S1 = pd.read_excel('pancancer1.xlsx')
cosmic = pd.read_excel('COSMIC.xlsx')

In [107]:
# merge dataframes
cosmic["Ensembl gene id"] = cosmic.Ensembl_gene_ID
complete_data = cosmic.join(S1.set_index("Ensembl gene id"), "Ensembl gene id", how="left")

#clean up data and creating working dataframe 
delete = ['Ensembl gene id', 'Chromosome', 'Start_position', 'Variant id', 'End_position', 'Annotation_sites', 'Uniprots id', 'Gene symbol']
working_data = complete_data.drop(columns = delete) 

In [170]:
# create a numerical index for classifying base pair substitutions
bp_sub = [0] * working_data.shape[0]
# A, T, G, C
#:647872
for i, r, in working_data.head(50).iterrows():
    print( r["Reference_nucleotide"] ,r['Altered_nucleotide'])
for i,ref in enumerate(working_data[0:37]['Reference_nucleotide']):
    print(str(ref), str(working_data.loc[i, 'Altered_nucleotide']))
    if str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 0
    elif str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 1
    elif str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 2
    elif str(ref) == 'A' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 3
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 4
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 0
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 5
    elif str(ref) == 'T' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 6
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 7
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 8
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 0
    elif str(ref) == 'G' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 9
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'A':
        bp_sub[i] = 10
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'T':
        bp_sub[i] = 11
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'G':
        bp_sub[i] = 12
    elif str(ref) == 'C' and str(working_data.loc[i, 'Altered_nucleotide']) == 'C':
        bp_sub[i] = 0
working_data['Base Pair Substitutions'] = bp_sub

G T
T A
G A
G T
G A
G T
G A
T G
A G
G C
G A
C G
G A
G A
A G
G C
T C
A T
G A
A G
C A
G A
C T
G A
C T
C T
C T
G A
C T
G A
A G
G A
C G
G A
G T
A G
C T
C T
C T
C T
T C
C T
G T
A G
A T
G T
C T
A T
G T
G T
G T
T A
G A
G T
G A
G T
G A
T G
A G
G C
G A
C G
G A
G A
A G
G C
T C
A T
G A
A G
C A
G A
C T
G A
C T
C T
C T
G A
C T
G A
A G
G A
C G
G A
G T
A G
C 36    T
36    T
36    T
36    T
Name: Altered_nucleotide, dtype: object


In [153]:
working_data.loc[153024]

Reference_nucleotide                            T
Altered_nucleotide                              C
Ensembl_gene_ID                   ENSG00000269871
Amino_acid_substitution                      L51P
PON-P2_predicted_probability                0.428
PON-P2_error                                0.108
PON-P2_classification                     Unknown
Frequency_in_COSMIC                             1
Neoantigen freq                               NaN
Variant freq                                  NaN
Protein length                                NaN
Length-normalized variant freq                NaN
Base Pair Substitutions                         0
Pathogenicity Classification                    0
Name: 153024, dtype: object

In [137]:
# create a numberical index for pathogenicity classification
pathogenicity_classification = []
for pathogenicity in working_data['PON-P2_classification']:
    if str(pathogenicity) == 'Pathogenic':
        pathogenicity_classification.append(1)
    else:
        pathogenicity_classification.append(0)
working_data['Pathogenicity Classification'] = pathogenicity_classification

In [141]:
# Base-Pair Sub Key
key = pd.DataFrame(data = ['None','A to T', 'A to G', 'A to C', 'T to A', 'T to G', 'T to C', 'G to A', 'G to T', 'G to C', 'C to A', 'C to T', 'C to G'], columns = ['Base Pair Substitution'], index  = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
key['Pathogenicity'] = ['Pathogenic', 'Neutral/Unknown', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']

In [151]:
numerical_data = working_data.drop(columns = ['Reference_nucleotide', 'Altered_nucleotide', 'PON-P2_classification'])
working_data.sort_values(by = 'Ensembl_gene_ID', ascending = False)

,Reference_nucleotide,Altered_nucleotide,Ensembl_gene_ID,Amino_acid_substitution,PON-P2_predicted_probability,PON-P2_error,PON-P2_classification,Frequency_in_COSMIC,Neoantigen freq,Variant freq,Protein length,Length-normalized variant freq,Base Pair Substitutions,Pathogenicity Classification
153024,T,C,ENSG00000269871,L51P,0.428,0.108,Unknown,1,NaN,NaN,NaN,NaN,0,0
153025,G,T,ENSG00000269871,R147L,0.200,0.060,Neutral,1,NaN,NaN,NaN,NaN,7,0
153026,A,G,ENSG00000269871,E72G,0.093,0.044,Neutral,1,NaN,NaN,NaN,NaN,1,0
151177,G,T,ENSG00000269858,W242C,0.872,0.062,Pathogenic,1,202.0,34.0,407.0,0.083538,9,1
151182,G,A,ENSG00000269858,G293R,0.864,0.090,Unknown,1,202.0,34.0,407.0,0.083538,7,0
151180,C,T,ENSG00000269858,R187W,0.888,0.066,Pathogenic,1,202.0,34.0,407.0,0.083538,10,1
151179,G,A,ENSG00000269858,V336M,0.413,0.107,Unknown,1,202.0,34.0,407.0,0.083538,7,0
151178,G,T,ENSG00000269858,A173S,0.172,0.057,Neutral,1,202.0,34.0,407.0,0.083538,7,0
151183,G,T,ENSG00000269858,C307F,0.331,0.056,Unknown,1,202.0,34.0,407.0,0.083538,8,0
151176,G,T,ENSG00000269858,G303V,0.693,0.072,Unknown,1,202.0,34.0,407.0,0.083538,8,0


,Ensembl_gene_ID,Amino_acid_substitution,PON-P2_predicted_probability,PON-P2_error,Frequency_in_COSMIC,Neoantigen freq,Variant freq,Protein length,Length-normalized variant freq,Base Pair Substitutions,Pathogenicity Classification
153024,ENSG00000269871,L51P,0.428,0.108,1,NaN,NaN,NaN,NaN,0,0
153025,ENSG00000269871,R147L,0.200,0.060,1,NaN,NaN,NaN,NaN,7,0
153026,ENSG00000269871,E72G,0.093,0.044,1,NaN,NaN,NaN,NaN,1,0
151177,ENSG00000269858,W242C,0.872,0.062,1,202.0,34.0,407.0,0.083538,9,1
151182,ENSG00000269858,G293R,0.864,0.090,1,202.0,34.0,407.0,0.083538,7,0
151180,ENSG00000269858,R187W,0.888,0.066,1,202.0,34.0,407.0,0.083538,10,1
151179,ENSG00000269858,V336M,0.413,0.107,1,202.0,34.0,407.0,0.083538,7,0
151178,ENSG00000269858,A173S,0.172,0.057,1,202.0,34.0,407.0,0.083538,7,0
151183,ENSG00000269858,C307F,0.331,0.056,1,202.0,34.0,407.0,0.083538,8,0
151176,ENSG00000269858,G303V,0.693,0.072,1,202.0,34.0,407.0,0.083538,8,0
